# Lab 8: Regression

In [ ]:
!pip install pingouin --upgrade

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import matplotlib.pyplot as plt
import pingouin as pg
import statsmodels.api as sm
pd.options.display.float_format = '{:.4f}'.format

In [ ]:
#Show our plots in the Jupyter notebook
%matplotlib inline

In [ ]:
#When we start working with nan (missing) values, we can get warnings - we're going to ignore them here
#You might not want to do this the first time around to see "under the hood" a bit more
import warnings
warnings.filterwarnings("ignore") 

## 1.0 Bring .csv into Python

In [ ]:
df=pd.read_csv('Final_CP201Asurveydata.csv', delimiter = ',')

In [ ]:
df

## 2. Cleaning and recoding data

You should be familiar with the codes below. I'm just going to go through the steps, without any explanatory text.

In [ ]:
df.rename(columns={'Survey Number': 'survey_num', 
                        'Day of the Week': 'day_of_week', 
                        'Time of Day': 'time',
                        'Interviewer or self-administered': 'collection_mode',
                        'Connection to neighborhood': 'connection_nbhd',
                        'Average days spent in neighborhood per week': 'days_week',
                        'Neighborhood recovered from pandemic': 'pandemic_recover',
                        'Safe walking alone at night': 'safe_walking',
                        'New housing has increased gentrification': 'gentrification',
                        'Bikes lanes have increased traffic safety': 'bike_lane',
                        'I feel connected to the people and community in this city.': 'connected',
                        'Support increasing the supply of housing': 'housing_supply',
                        'Support speed enforcement cameras': 'speed_camera',
                        'Support sales tax to fund climate disaster preparedness': 'climate_tax',
                        'Support investment in policing': 'policing', 
                        'Gender identity': "gender_identity",
                        'Rent or own': 'tenure',
                        'Biggest challenge facing city': 'biggest_challenge',
                       'Race/ethnicity':'race_eth',
                        'Notes (include data here on "Other" responses, observational notes.)': 'notes'}, inplace=True)

### 2.1 Create outcome (dependent) variable: Housing supply numeric and dummy

In [ ]:
df[['housing_supply']].value_counts(dropna=False)

In [ ]:
#supply numeric - higher value = agree more strongly
df['supplyagree_numeric']=df['housing_supply'].map({"Strongly Agree":5, "Agree": 4, "Neutral": 3,
                                                                   "Disagree":2, "Strongly Disagree": 1, "Don't Know/NA":0})

In [ ]:
#supply dummy - agree =1, else 0
df['supplyagree_dv']=df['housing_supply'].map({"Strongly Agree":1, "Agree": 1, "Neutral": 0,
                                                                   "Disagree":0, "Strongly Disagree": 0, "Don't Know/NA":0})

### 2.2 Create explanatory (independent) variables: Tenure, Age, Gender, and Gentrification

In [ ]:
#owner dummy variable
df['own_dv']=df['tenure'].map({"Own": 1, "Other":1, "Rent":0})

In [ ]:
#male dummy variable
df['male_dv']=df['gender_identity'].map({"Male": 1, "Female":0, "Non-binary":0, "Other/Prefer not to answer":0})

In [ ]:
#age dummy variable - creating one for under 45
df['under45_dv']=df['Age'].map({"25-44 years old": 1, "45-64 years old":0, "65 or older":0, "Prefer not to answer":0, 'Under 25 years old':1})

In [ ]:
#gentrification - I'm going to construct this as numeric, to demonstrate correlation
#gentrification numeric - higher value = agree more strongly
df['gentrifyagree_numeric']=df['gentrification'].map({"Strongly Agree":5, "Agree": 4, "Neutral": 3,
                                                                   "Disagree":2, "Strongly Disagree": 1, "Don't Know/NA":0})

## 3 Correlation

The correlation coefficient (sometimes referred to as Pearson's correlation coefficient, Pearson's product-moment correlation, or simply r) measures the strength of the linear relationship between two variables. 

The correlation coefficient is directly linked to the beta coefficient in a linear regression (= the slope of a best-fit line), but has the advantage of being standardized between -1 to 1 ; the former meaning a perfect negative linear relationship, and the latter a perfect positive linear relationship. In other words, no matter what are the original units of the two variables are, the correlation coefficient will always be in the range of -1 to 1, which makes it very easy to work with.

The correlation coefficient *r*

> The correlation coefficient ranges from −1 to 1. A value of 1 implies that a linear equation describes the relationship between X and Y perfectly, with all data points lying on a line for which Y increases as X increases. A value of −1 implies that all data points lie on a line for which Y decreases as X increases. A value of 0 implies that there is no linear relationship between the variables. 

In hypothesis testing, you want to find not only the correlation coefficient (the r value) but also the p-value.

In [ ]:
#Let's start by plotting the two numeric variables
sns.regplot(
    x=df["gentrifyagree_numeric"],  
    y=df["supplyagree_numeric"],   
    ci=None, 
    scatter_kws={"color": "black"}, 
    line_kws={"color": "red"}
)

plt.xlabel("Agrees New Housing Supply Spurs Gentrification")
plt.ylabel("Agrees Need for More Housing Supply")
plt.title("Regression Plot")
plt.show()

In [ ]:
#Let's run our correlation test
pg.corr(x=df['gentrifyagree_numeric'], y=df["supplyagree_numeric"])

Not suprising that we don't see a very large r or significant p-value - there's almost no slope to the line

## 4. Linear Regression

Linear regression is just an extension of correlation.  Let's take a look!

In [ ]:
import statsmodels.formula.api as smf

# List of independent variables
ind_vars = ['own_dv', 'male_dv']

# Build the formula string automatically
formula = 'supplyagree_numeric ~ ' + ' + '.join(ind_vars)

# Run regression (missing='drop' handles NaNs)
model = smf.ols(formula=formula, data=df, missing='drop').fit()

print(model.summary())

Cool!  We find that even after controlling for the male_dv (which is not significant, suggesting that male respondents did not agree with the need for more housing supply than those who did not identify as male), owners are less likely (negative coefficient, and significant) to support new housing supply.  In fact, being an owner reduces the "agree intensity score" by .35.  My R-squared is pretty low though - I'm just explaining 2% of the variation in whether someone agrees with the need for more housing supply.

What happens when I add  'under45_dv', 'gentrifyagree_numeric' to my independent variables?

## 4.2 Logit Regression

Logit regression is used when your dependent variable is a dummy, not numeric, variable. Explanatory variables can be of any type.

In [ ]:
# Run the logit model; here, we put the dependent variable before the squiggly character, and list all our explantory variables
#after the squiggly character
logit_model = smf.logit(
    formula="supplyagree_dv ~ own_dv + male_dv",
    data=df
).fit()

#summary2 here is just part of the package specification - summary() gives you an older version of the output
logit_model.summary2()

In [ ]:
# then, we calculate the odds ratios 
or_table = np.exp(logit_model.conf_int()) #Exponentiate Confidence Intervals
or_table['Odds Ratio'] = np.exp(logit_model.params) #Exponentiate Coefficients

or_table.columns = ['5%', '95%', 'Odds Ratio'] #Name Columns
or_table

Unlike linear regression, where we can interpret the results as "if x goes up by one unit, the response of Y is xx", in logistic, we calculate the odds ratio.  Although the male_dv variable is not significant, if it were, I could interpret the odds ratio as "Men are 20% more likely to support additional housing supply than those who didn't identify as men."  For own, which is significant, I can interpret it as "Owners were about half as likely to support new housing supply (1-.52), even after controlling for gender. 

Play around with it! Developing a model is more art than science - we can look at your output together in Lab and discuss how to interpret!  But in the end, it's all about small p-values!